# Pruebas para resolver un PL con Pulp:
Estoy sacando el ejemplo de: http://benalexkeen.com/linear-programming-with-python-and-pulp-part-2/

In [20]:
import pulp
import numpy as np
%matplotlib inline

Creando el problema "MiPL" e indicando que es de maximización

In [239]:
mi_modelo_lp = pulp.LpProblem("Mi PL", pulp.LpMaximize)

El problema es
Maximizar Z = 4x+3y

Sujeto a:
x≥0
y≥2
2y≤25–x
4y≥2x–8
y≤2x−5

al definir las variables, de una vez indico su limite inferior

In [240]:
x = pulp.LpVariable('x', lowBound = 0, cat = 'Continuous')
y = pulp.LpVariable('y', lowBound = 2, cat = 'Continuous')

Se agrega la funcion objetivo al modelo usando el operador += , de igual forma se procede con las restricciones.

In [241]:
# Funcion Objetivo
mi_modelo_lp += + 4 * x + 3 * y, "Z"

# Restricciones
mi_modelo_lp += 2 * y <= 25 - x
mi_modelo_lp += 4 * y >= 2 * x - 8
mi_modelo_lp += y <= 2 * x - 5

In [238]:
mi_modelo_lp

Mi PL:
MAXIMIZE
4*x + 3*y + 0
SUBJECT TO
_C1: x + 2 y <= 25

_C2: - 2 x + 4 y >= -8

_C3: - 2 x + y <= -5

VARIABLES
x Continuous
2 <= y Continuous

El solver por defecto es CBC, que viene con la instalacion de PuLP. COIN-OR es otro que probablemente resuelve la mayoria de los LPs

In [15]:
mi_modelo_lp.solve()

1

In [16]:
pulp.LpStatus[mi_modelo_lp.status]

'Optimal'

Se puede usar el metodo varValue para recuperar los valores de las variables x y y, y la funcion pulp.value para ver el maximo alcanzado por la funcion objetivo

In [18]:
for variable in mi_modelo_lp.variables():
    print "{}={}".format(variable.name, variable.varValue)

x=14.5
y=5.25


In [19]:
print pulp.value(mi_modelo_lp.objective)

73.75


# Ahora usando Matrices:
Voy a definir un problema pequeño en forma matricial, de la forma:

min c.x

sujeto a:

Ax >= b

x>= 0

Nota: use comandos de los scripts que se encuentran en:

http://stackoverflow.com/questions/7728313/python-pulp-using-with-matrices

https://github.com/AnnaNican/optimizers/blob/master/ReadBestsellers.ipynb

https://scaron.info/blog/linear-programming-in-python-with-pulp.html

http://stackoverflow.com/questions/22392497/how-to-add-a-new-row-to-an-empty-numpy-array

http://stackoverflow.com/questions/568962/how-do-i-create-an-empty-array-matrix-in-numpy

http://www.optimization-online.org/DB_FILE/2011/09/3178.pdf

https://groups.google.com/forum/#!topic/pulp-or-discuss/JIlUGNUNy44

In [34]:
# Matriz de restricciones
A = np.array([[1,2], [-1,1]])
print('Matriz de restricciones')
print(A)

Matriz de restricciones
[[ 1  2]
 [-1  1]]


In [209]:
# Vector de costos
c = np.array([1, 4])
print('Vector de Costos')
print(c)

Vector de Costos
[1 4]


In [129]:
# Vector recurso
b = np.array([[1],[2]])
print('Vector recurso')
print(b)

Vector recurso
[[1]
 [2]]


In [205]:
#Vector de variables (OPCION si no se usan "dicts")
#x = []
#for i in range(1,3):
#    x.append(['x'+str(i)])
#x = np.asarray(x)
#print('Nombres de variables')    
#print(x)

Defino el problema como uno de minimizacion

In [299]:
lp_mat = pulp.LpProblem("LP_matricial", pulp.LpMinimize)

Defino las variables opcionalmente usando la funcionalidad de "dicts"

In [300]:
decision_variables = []
x_min = 0
x_max = None
x = pulp.LpVariable.dicts("x", range(2), x_min, x_max) #por defecto se asume continua
print(x)
for i in range(1,len(x)+1):
    decision_variables.append(x[i-1])

print ("Total number of decision_variables: " + str(len(decision_variables)))
print ("Array with Decision Variables:" + str(decision_variables))

{0: x_0, 1: x_1}
Total number of decision_variables: 2
Array with Decision Variables:[x_0, x_1]


Ahora defino la funcion objetivo, usando el vector "c"

In [301]:
# Crear funcion objetivo
fo = pulp.lpDot(c,decision_variables) #sumaproducto de c y las variables
lp_mat += fo
print("Funcion Objetivo: Z= "+str(fo))


Funcion Objetivo: Z= x_0 + 4*x_1


Defino las restricciones usando la matriz A, las variables definidas y el vector b

In [302]:
# Crear restricciones
print('Restricciones:')
for coeff, rhs in zip(A, b):
    print(pulp.lpDot(coeff, decision_variables) >= rhs)
    lp_mat += pulp.lpDot(coeff, decision_variables) >= rhs


Restricciones:
x_0 + 2*x_1 >= 1
-x_0 + x_1 >= 2


In [303]:
print('Modelo cargado en PuLP:')
print(lp_mat)

Modelo cargado en PuLP:
LP_matricial:
MINIMIZE
1*x_0 + 4*x_1 + 0
SUBJECT TO
_C1: x_0 + 2 x_1 >= 1

_C2: - x_0 + x_1 >= 2

VARIABLES
x_0 Continuous
x_1 Continuous



Guardamos el modelo en formato de modelo PuLP

In [306]:
lp_mat.writeLP("lp_mat.lp" )

Finalmente.... la optimizacion del modelo:

In [314]:
#Correr el solver para optimizar el modelo
optimization_result = lp_mat.solve()
assert optimization_result == pulp.LpStatusOptimal
print("Status:", pulp.LpStatus[lp_mat.status])
print("Optimal Solution to the problem: ", pulp.value(lp_mat.objective))
print ("Individual decision_variables: ")
for v in lp_mat.variables():
	print(v.name, "=", v.varValue)



('Status:', 'Optimal')
('Optimal Solution to the problem: ', 8.0)
Individual decision_variables: 
('x_0', '=', 0.0)
('x_1', '=', 2.0)
